In [1]:
# Lab 10-1 MNIST CNN
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True, 
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/', 
                         train = False, 
                         transform = transforms.ToTensor(), 
                         download = True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          drop_last = True)

In [9]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    
    # L1 Image shape = (?, 28, 28, 1)
    #       conv    -> (?, 28, 28, 32)
    #       pool    -> (?, 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # L2 Image shape = (?, 14, 14, 32)
    #      conv     -> (?, 14, 14, 64)
    #      pool     -> (?, 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # Final FC 7X7X64 inputs -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)    # Flatten them for FC
    out = self.fc(out)
    return out



In [11]:
# instantiate CNN model
model = CNN().to(device)

In [12]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
# train my model
total_batch = len(data_loader)
print('Learning started. it takes sometime.')

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # image is aleady size of (28x28), no reshape
    # label is not one-hot encoded
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished!')

Learning started. it takes sometime.
[Epoch:    1] cost = 0.173929989
[Epoch:    2] cost = 0.0730266124
[Epoch:    3] cost = 0.0602604672
[Epoch:    4] cost = 0.0541095063
[Epoch:    5] cost = 0.0498446859
[Epoch:    6] cost = 0.0494956747
[Epoch:    7] cost = 0.0456784181
[Epoch:    8] cost = 0.0451378562
[Epoch:    9] cost = 0.0436725095
[Epoch:   10] cost = 0.0394681543
[Epoch:   11] cost = 0.0393189155
[Epoch:   12] cost = 0.0380259492
[Epoch:   13] cost = 0.0362511761
[Epoch:   14] cost = 0.0357399993
[Epoch:   15] cost = 0.0328910723
Learning Finished!


In [14]:
# Test model and check accuracy
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()

  print('Accuracy:', accuracy.item())

Accuracy: 0.9103999733924866


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
